In [ ]:
from sympy import *
from sympy.physics.mechanics import *
import sympy.physics.mechanics as me
me.init_vprinting()

In [ ]:
t, theta = symbols("t, theta")
phi, psi = dynamicsymbols("phi, psi")
phid, psid = dynamicsymbols("phi, psi", 1)
phidd, psidd = dynamicsymbols("phi, psi", 2)
omega_p, omega_s = dynamicsymbols("omega_p, omega_s")

# axial torque applied at D
T_D = dynamicsymbols("T_D")
# reaction forces acting on point D, E
D_forces = dynamicsymbols("D_x, D_y, D_z")
E_forces = dynamicsymbols("E_x, E_y")

# N: inertial frame
# M: frame of the rotating platform
# R: intermediate frame
# S: frame of the shaft
N, M, R, S = symbols("N, M, R, S", cls=ReferenceFrame)
M.orient_axis(N, N.z, phi)
R.orient_axis(M, M.x, theta)
R.orient_body_fixed(M, [theta, pi/2, 0], 231)
S.orient_axis(R, R.z, psi)

In [ ]:
m, b, h, l = symbols("m, b, h, l")
O, G, D, E = symbols("O, G, D, E", cls=Point)
G.set_pos(O, 2 * N.z)
D.set_pos(G, -l / 2 * S.z)
E.set_pos(G, l / 2 * S.z)

O.set_vel(N, 0)
G.set_vel(M, 0)
G.set_vel(R, 0)
G.set_vel(S, 0)
D.set_vel(M, 0)
E.set_vel(M, 0)
G.v2pt_theory(O, N, M)
G.a2pt_theory(O, N, M)
D.v2pt_theory(G, N, M)
D.a2pt_theory(G, N, M)
E.v2pt_theory(G, N, M)
E.a2pt_theory(G, N, M)

In [ ]:
def build_vector(components, frame):
    return sum(c * v for c, v in zip(components, frame))

forces_list = [
    (D, build_vector(D_forces, S)),
    (E, build_vector(E_forces, S)),
]
    
lhs = sum(e[1] for e in forces_list)
rhs = m * G.acc(N)
Force_eq = lhs - rhs
Force_eq

In [ ]:
D_sol = solve(Force_eq.to_matrix(S), D_forces)
D_sol

In [ ]:
Moment_G_eq = sum(point.pos_from(G) ^ force for point, force in forces_list) + T_D * S.z
Moment_G_eq

In [ ]:
Moment_G_eq = Moment_G_eq.subs(D_sol)
Moment_G_eq

In [ ]:
A, B, C = symbols("A:C")
I_G = inertia(S, A, B, C)
shaft = RigidBody("shaft", G, S, m, (I_G, G))
H_G = shaft.angular_momentum(G, N)
H_G

In [ ]:
rhs = H_G.diff(t, S) + (S.ang_vel_in(N) ^ H_G)
euler = Moment_G_eq - rhs
euler

In [ ]:
assumptions = {
    phidd: 0, psidd: 0,
    phid: omega_p, psid: omega_s
}
sd_inertia = {
    A: m * (h**2 + l**2) / 12,
    B: m * (b**2 + l**2) / 12,
    C: m * (b**2 + h**2) / 12,
}
euler = euler.subs(assumptions).subs(sd_inertia).to_matrix(S)
unknowns = [*E_forces, T_D]
sol = solve(euler, unknowns)
e1, e2, e3 = sol.values()
e1 = e1.simplify().collect(omega_p * cos(theta))
e2 = e2.simplify().collect(omega_p * cos(theta))
e3 = e3.simplify()
display(*[Eq(f, e) for f, e in zip(unknowns, [e1, e2, e3])])

In [ ]:
# Plotting results

sd = {
    l: 1,           # m
    h: 0.1,         # m
    b: 0.025,       # m
    theta: pi/6,    # rad
    m: 10,          # kg
    omega_p: 10.47, # rad/s
    omega_s: 209.4, # rad/s
}
V = sqrt(solutions[0]**2 + solutions[1]**2)

from spb import plot
import numpy as np
plot(V.subs(sd), (psi, 0, 2*pi), tx=np.degrees, xlabel=r"$\psi$ [deg]", ylabel="V [N]")

In [ ]:
plot(solutions[2].subs(sd), (psi, 0, 2*pi), tx=np.degrees, xlabel=r"$\psi$ [deg]", ylabel="$T_{D}$ [Nm]")